In [0]:
%restart_python
%pip install boto3
import boto3
import os
from botocore.exceptions import NoCredentialsError
import datetime
import sys
sys.path.insert(0, '/Workspace/Shared')
import etl_helpers
from pyspark.sql.functions import col #lit
# from pyspark.sql.types import StringType #IntegerType, BooleanType

tablename = "dimaddress"
runcycleid = etl_helpers.start_run_cycle(tablename)
os.makedirs("/dbfs/foi/dataload", exist_ok=True)  # make sure directory exists

try:

    df_lastrun = spark.sql(f"SELECT runcyclestartat as createddate FROM dimruncycle WHERE packagename = \"{tablename}\"  AND success = 't' ORDER BY runcycleid DESC LIMIT 1")
    
    if df_lastrun.count() > 0:
        lastruntime = df_lastrun.first().createddate.strftime("%Y-%m-%d %H:%M:%S")
    else:
        lastruntime = "2019-01-01 00:00:00"
    print(lastruntime)

    query = f"""
        SELECT
            t.foirequest_id AS foirequestid,
            MAX(CASE WHEN t.dataformat = 'email' THEN t.contactinformation END) AS email,
            MAX(CASE WHEN t.dataformat = 'country' THEN t.contactinformation END) AS country,
            MAX(CASE WHEN t.dataformat = 'postal' THEN t.contactinformation END) AS postal,
            MAX(CASE WHEN t.dataformat = 'province' THEN t.contactinformation END) AS province,
            MAX(CASE WHEN t.dataformat = 'city' THEN t.contactinformation END) AS city,
            MAX(CASE WHEN t.dataformat = 'address' THEN t.contactinformation END) AS address1,
            MAX(CASE WHEN t.dataformat = 'addressSecondary' THEN t.contactinformation END) AS address2,
            MAX(t.foirequestversion_id) AS max_foirequestversion_id,
            'FOIMOD' AS sourceoftruth
        FROM
            foi_mod.foirequestcontactinformation t
        INNER JOIN (
            -- Subquery to find the maximum foirequestversion_id for each foirequest_id
            SELECT
                foirequest_id,
                MAX(foirequestversion_id) AS latest_version
            FROM
                foi_mod.foirequestcontactinformation
            GROUP BY
                foirequest_id
        ) AS latest_versions ON t.foirequest_id = latest_versions.foirequest_id
                            AND t.foirequestversion_id = latest_versions.latest_version
        WHERE
            t.dataformat IN ('email', 'country', 'postal', 'province', 'city', 'address', 'addressSecondary')
            and (t.created_at > '{lastruntime}' or t.updated_at > '{lastruntime}')
        GROUP BY
            t.foirequest_id
        ORDER BY
            t.foirequest_id desc    
        """

    # print(query)

    df = spark.sql(query)
    df.show()

    # order of columns here is important!
    df_mapped = df.selectExpr(
            "address1 AS address1",
            "address2 AS address2",
            "city AS city",
            "province AS state",
            "country AS country",
            # "email AS email",
            "postal AS zipcode",
            "foirequestid AS foirequestid",
            # f"{runcycleid} as runcycleid",
            "sourceoftruth AS sourceoftruth"
        )
    df_mapped.show()

    from delta.tables import DeltaTable
    delta_table = DeltaTable.forName(spark, f"hive_metastore.default.{tablename}")
    delta_table.alias("target").merge(
        df_mapped.alias("source"),
        "target.foirequestid = source.foirequestid AND target.sourceoftruth = source.sourceoftruth"
    ).whenMatchedUpdate(set = {
        "address1": col("source.address1"),
        "address2": col("source.address2"),
        "city": col("source.city"),
        "state": col("source.state"),
        "country": col("source.country"),
        "zipcode": col("source.zipcode")
    }).whenNotMatchedInsert(values = {
        "address1": col("source.address1"),
        "address2": col("source.address2"),
        "city": col("source.city"),
        "state": col("source.state"),
        "country": col("source.country"),
        "zipcode": col("source.zipcode"),
        "foirequestid": col("source.foirequestid"),
        "sourceoftruth": col("source.sourceoftruth")
    }).execute()

    etl_helpers.end_run_cycle(runcycleid, 't', tablename)
except NoCredentialsError:
    print("Credentials not available")
    etl_helpers.end_run_cycle(runcycleid, 'f', tablename, "Credentials not available")
except Exception as e:
    if (str(e) == "no changes for today"):
        print("here")
        etl_helpers.end_run_cycle(runcycleid, 't', tablename)
    else:
        print(f"An error occurred: {e}")    
        etl_helpers.end_run_cycle(runcycleid, 'f', tablename, f"An error occurred: {e}")